In [2]:
import imageio
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from ptflops import get_model_complexity_info

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\ProgramData\Anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
##############################################################################################################################
#########Problem 1#########
##############################################################################################################################

In [3]:
import pandas as pd
housing = pd.DataFrame(pd.read_csv("Housing.csv"))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price'] 
Newtrain = housing[num_vars] 
Newtrain.head() 

,area,bedrooms,bathrooms,stories,parking,price
0,7420,4,2,3,2,13300000
1,8960,4,4,4,3,12250000
2,9960,3,2,2,2,12250000
3,7500,4,2,2,3,12215000
4,7420,4,1,2,2,11410000


In [5]:
#Price Predicition
t_area = torch.tensor(Newtrain['area']) 
t_bedrooms = torch.tensor(Newtrain['bedrooms'])
t_bathrooms = torch.tensor(Newtrain['bathrooms'])
t_stories = torch.tensor(Newtrain['stories'])
t_parking = torch.tensor(Newtrain['parking'])
t_price = torch.tensor(Newtrain['price'])

#Normalized #Split Post Normalized Values
t_areaN = t_area / max(Newtrain['area'])
t_bedroomsN = t_bedrooms / max(Newtrain['bedrooms'])
t_bathroomsN = t_bathrooms / max(Newtrain['bathrooms'])
t_storiesN = t_stories / max(Newtrain['stories'])
t_parkingN = t_parking / max(Newtrain['parking'])
t_priceN = t_price / max(Newtrain['price'])

#Split ^
#Mask Creation 80/20 Split
mask = np.random.rand(len(t_areaN)) <= 0.8

#AreaSets
t_area_train = t_areaN[mask]
t_area_valid = t_areaN[~mask]

#BedroomSets
t_bedrooms_train = t_bedroomsN[mask]
t_bedrooms_valid = t_bedroomsN[~mask]

#BathroomSets
t_bathrooms_train = t_bathroomsN[mask]
t_bathrooms_valid = t_bathroomsN[~mask]

#StoriesSets
t_stories_train = t_storiesN[mask]
t_stories_valid = t_storiesN[~mask]

#ParkingSets
t_parking_train = t_parkingN[mask]
t_parking_valid = t_parkingN[~mask]

#PriceSets
t_price_train = t_priceN[mask]
t_price_valid = t_priceN[~mask]

In [6]:
#Bringing the normalized features back together and making it a tensor.
training_features = [t_area_train, t_bedrooms_train, t_bathrooms_train, t_stories_train, t_parking_train]
training_features = torch.stack(training_features)
training_features = np.transpose(training_features)

validation_features = [t_area_valid, t_bedrooms_valid, t_bathrooms_valid, t_stories_valid, t_parking_valid]
validation_features = torch.stack(validation_features)
validation_features = np.transpose(validation_features)


In [7]:
training_features

tensor([[0.4580, 0.6667, 0.5000, 0.7500, 0.6667],
        [0.5531, 0.6667, 1.0000, 1.0000, 1.0000],
        [0.6148, 0.5000, 0.5000, 0.5000, 0.6667],
        ...,
        [0.1481, 0.5000, 0.2500, 0.2500, 0.0000],
        [0.1796, 0.5000, 0.2500, 0.2500, 0.0000],
        [0.2377, 0.5000, 0.2500, 0.5000, 0.0000]])

In [8]:
validation_features

tensor([[1.0000, 0.8333, 0.7500, 0.5000, 0.0000],
        [0.8148, 0.5000, 0.2500, 0.5000, 0.6667],
        [0.3704, 0.6667, 0.7500, 0.5000, 0.6667],
        [0.5247, 0.5000, 0.5000, 1.0000, 0.6667],
        [0.3704, 0.5000, 0.5000, 1.0000, 0.0000],
        [0.3679, 0.5000, 0.7500, 0.5000, 0.3333],
        [0.5556, 0.6667, 0.5000, 1.0000, 0.6667],
        [0.3704, 0.6667, 0.5000, 1.0000, 0.3333],
        [0.5556, 0.6667, 0.5000, 1.0000, 0.3333],
        [0.3704, 0.5000, 0.5000, 0.5000, 0.3333],
        [0.3852, 0.6667, 0.5000, 0.5000, 0.3333],
        [0.5481, 0.5000, 0.5000, 0.5000, 0.3333],
        [0.7463, 0.6667, 0.5000, 0.5000, 0.6667],
        [0.3704, 0.6667, 0.5000, 1.0000, 0.0000],
        [0.3099, 0.5000, 0.2500, 1.0000, 0.0000],
        [0.4012, 0.5000, 0.5000, 0.7500, 0.0000],
        [0.2444, 0.5000, 0.2500, 0.2500, 0.6667],
        [0.5296, 0.8333, 0.7500, 0.5000, 0.6667],
        [0.3704, 0.6667, 0.5000, 1.0000, 0.3333],
        [0.5168, 0.5000, 0.2500, 0.7500, 0.6667],


In [5]:
# Sequential Model Setup 
import torch.nn as nn
import torch.optim as optim
hw2_seq_model = nn.Sequential(
                nn.Linear(5,8), # (X,Y) X = Input Features Y = Output Features
                nn.Tanh(),  # Neuron Activiation Function
                nn.Linear(8,16), # Hidden Layer 
                nn.Tanh(),
                nn.Linear(16,1)) # End Layer (Y,N) # Output Features as Input, N - Price Output
hw2_seq_model


Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=16, bias=True)
  (3): Tanh()
  (4): Linear(in_features=16, out_features=1, bias=True)
)

In [10]:
#Optimizer Initialization
OPTIM = optim.SGD(hw2_seq_model.parameters(), lr = 1e-3)

In [11]:
#Training Loop Function Declaration
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():0.4f},"f" Validation loss {loss_val.item():.4f}")
            

In [12]:
#Training Loop Intialization
training_loop(
    n_epochs = 200,
    optimizer = OPTIM,
    model = hw2_seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = training_features,
    t_u_val = validation_features,
    t_c_train = t_price_train,
    t_c_val = t_price_valid)

Epoch 1, Training loss 0.1987, Validation loss 0.1911
Epoch 100, Training loss 0.0762, Validation loss 0.0711
Epoch 200, Training loss 0.0383, Validation loss 0.0344


C:\Users\colef\.conda\envs\gpu2\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([440])) that is different to the input size (torch.Size([440, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\colef\.conda\envs\gpu2\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([105])) that is different to the input size (torch.Size([105, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [13]:
with torch.cuda.device(0):
  net = hw2_seq_model
  macs, params = get_model_complexity_info(net, (1, 5), as_strings=True, print_per_layer_stat=True, 
verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 22.967% Params, 0.0 GMac, 22.967% MACs, in_features=5, out_features=8, bias=True)
  (1): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (2): Linear(0.0 M, 68.900% Params, 0.0 GMac, 68.900% MACs, in_features=8, out_features=16, bias=True)
  (3): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (4): Linear(0.0 M, 8.134% Params, 0.0 GMac, 8.134% MACs, in_features=16, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           209     


In [14]:
##############################################################################################################################
#########Problem 1 Part 2#########
##############################################################################################################################

In [15]:
# Sequential Model Setup 
hw2_seq_model2 = nn.Sequential(
                nn.Linear(5,8), # (X,Y) X = Input Features Y = Output Features
                nn.Tanh(),  # Neuron Activiation Function
                nn.Linear(8,16), # Hidden Layer 
                nn.Tanh(),
                nn.Linear(16, 32),
                nn.Tanh(),
                nn.Linear(32,64),
                nn.Tanh(),
                nn.Linear(64,1)) # End Layer (Y,N) # Output Features as Input, N - Price Output
hw2_seq_model2

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=16, bias=True)
  (3): Tanh()
  (4): Linear(in_features=16, out_features=32, bias=True)
  (5): Tanh()
  (6): Linear(in_features=32, out_features=64, bias=True)
  (7): Tanh()
  (8): Linear(in_features=64, out_features=1, bias=True)
)

In [16]:
#Optimizer Initialization
OPTIM2 = optim.SGD(hw2_seq_model2.parameters(), lr = 1e-3)

In [17]:
#Training Loop Function Declaration
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():0.4f},"f" Validation loss {loss_val.item():.4f}")
            

In [18]:
training_loop(
    n_epochs = 200,
    optimizer = OPTIM2,
    model = hw2_seq_model2,
    loss_fn = nn.MSELoss(),
    t_u_train = training_features,
    t_u_val = validation_features,
    t_c_train = t_price_train,
    t_c_val = t_price_valid)

Epoch 1, Training loss 0.2281, Validation loss 0.2189
Epoch 100, Training loss 0.0593, Validation loss 0.0540
Epoch 200, Training loss 0.0271, Validation loss 0.0233


In [19]:
with torch.cuda.device(0):
  net = hw2_seq_model2
  macs, params = get_model_complexity_info(net, (1, 5), as_strings=True, print_per_layer_stat=True, 
verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.003 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 1.648% Params, 0.0 GMac, 1.648% MACs, in_features=5, out_features=8, bias=True)
  (1): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (2): Linear(0.0 M, 4.943% Params, 0.0 GMac, 4.943% MACs, in_features=8, out_features=16, bias=True)
  (3): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (4): Linear(0.001 M, 18.675% Params, 0.0 GMac, 18.675% MACs, in_features=16, out_features=32, bias=True)
  (5): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (6): Linear(0.002 M, 72.503% Params, 0.0 GMac, 72.503% MACs, in_features=32, out_features=64, bias=True)
  (7): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (8): Linear(0.0 M, 2.231% Params, 0.0 GMac, 2.231% MACs, in_features=64, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           2.91 k  


In [20]:
##############################################################################################################################
#########Problem 2 Part 1#########
#############################################################################################################################

In [21]:
transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))

Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
from torchvision import datasets
data_path = 'C:/Users/colef/Documents/MachineLearningCIFAR10'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [24]:
CIFAR_model = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,10),
              nn.LogSoftmax(dim=1))
CIFAR_model.to(device)
loss_fn = nn.NLLLoss()

In [25]:
optimizer = optim.SGD(CIFAR_model.parameters(),lr=1e-3)

In [26]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
n_epochs = 300

In [27]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.614596
Epoch: 1, Loss: 2.117877
Epoch: 2, Loss: 1.805727
Epoch: 3, Loss: 1.566769
Epoch: 4, Loss: 1.955997
Epoch: 5, Loss: 1.614391
Epoch: 6, Loss: 1.915290
Epoch: 7, Loss: 1.498046
Epoch: 8, Loss: 2.015746
Epoch: 9, Loss: 1.725687
Epoch: 10, Loss: 2.360999
Epoch: 11, Loss: 2.358185
Epoch: 12, Loss: 1.637207
Epoch: 13, Loss: 2.048340
Epoch: 14, Loss: 1.133327
Epoch: 15, Loss: 1.956957
Epoch: 16, Loss: 1.942017
Epoch: 17, Loss: 1.783881
Epoch: 18, Loss: 1.290211
Epoch: 19, Loss: 1.652921
Epoch: 20, Loss: 1.640243
Epoch: 21, Loss: 1.627741
Epoch: 22, Loss: 1.876759
Epoch: 23, Loss: 1.864081
Epoch: 24, Loss: 1.928416
Epoch: 25, Loss: 1.620163
Epoch: 26, Loss: 1.380888
Epoch: 27, Loss: 1.782860
Epoch: 28, Loss: 1.259953
Epoch: 29, Loss: 1.362238
Epoch: 30, Loss: 1.502514
Epoch: 31, Loss: 2.192045
Epoch: 32, Loss: 1.560259
Epoch: 33, Loss: 1.853304
Epoch: 34, Loss: 1.586539
Epoch: 35, Loss: 1.254144
Epoch: 36, Loss: 1.741871
Epoch: 37, Loss: 1.325702
Epoch: 38, Loss: 1.738

In [31]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %f", (correct/total))

Accuracy: %f 0.4855


In [3]:
##############################################################################################################################
#########Problem 2 Part 2#########
#############################################################################################################################

In [13]:
CIFAR_model2 = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,256),
              nn.Tanh(),
              nn.Linear(256,128),
              nn.Tanh(),
              nn.Linear(128,10),
              nn.LogSoftmax(dim=1))
CIFAR_model2.to(device)
loss_fn = nn.NLLLoss()

In [14]:
optimizer = optim.SGD(CIFAR_model2.parameters(),lr=1e-2)

In [15]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
n_epochs = 300

In [16]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model2(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.169474
Epoch: 1, Loss: 1.832735
Epoch: 2, Loss: 1.932111
Epoch: 3, Loss: 1.646647
Epoch: 4, Loss: 2.064003
Epoch: 5, Loss: 1.475062
Epoch: 6, Loss: 1.326771
Epoch: 7, Loss: 1.258244
Epoch: 8, Loss: 1.374021
Epoch: 9, Loss: 1.543547
Epoch: 10, Loss: 1.605304
Epoch: 11, Loss: 1.248654
Epoch: 12, Loss: 1.517236
Epoch: 13, Loss: 1.652164
Epoch: 14, Loss: 1.247386
Epoch: 15, Loss: 0.845520
Epoch: 16, Loss: 0.879568
Epoch: 17, Loss: 1.217232
Epoch: 18, Loss: 1.432421
Epoch: 19, Loss: 1.257779
Epoch: 20, Loss: 0.790998
Epoch: 21, Loss: 0.764496
Epoch: 22, Loss: 0.950711
Epoch: 23, Loss: 0.819069
Epoch: 24, Loss: 1.061929
Epoch: 25, Loss: 0.718289
Epoch: 26, Loss: 0.389545
Epoch: 27, Loss: 0.413426
Epoch: 28, Loss: 0.448888
Epoch: 29, Loss: 0.308737
Epoch: 30, Loss: 0.781674
Epoch: 31, Loss: 0.430049
Epoch: 32, Loss: 0.850836
Epoch: 33, Loss: 0.309553
Epoch: 34, Loss: 0.638327
Epoch: 35, Loss: 0.361515
Epoch: 36, Loss: 0.324023
Epoch: 37, Loss: 0.562612
Epoch: 38, Loss: 0.195

In [17]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model2(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %f", (correct/total))

Accuracy: %f 0.4525
